# DURATION-NEUTRAL DTS-TARGET PORTFOLIO OPTIMIZATION

In [1]:
import pandas as pd
import numpy as np
from scipy.optimize import minimize
import pyfolio as pf
import polars as pl

import warnings
warnings.filterwarnings('ignore')

# Load trading data
data = pd.read_excel("Data/TradingData.xlsx")

# Load treasury data
Treasuries = pd.read_excel("Data/TradingTreasuries.xlsx")

# Load risk-free rate (RF) data
rf_data = pd.read_csv("Data/rf_constant.csv")
rf_data.rename(columns={"Week_Start": "Date"}, inplace=True)

# Format datetime columns
data['Date'] = pd.to_datetime(data['Date'])
data['maturity_date'] = pd.to_datetime(data['maturity_date'], errors='coerce')
data['next_call_date'] = pd.to_datetime(data['next_call_date'], errors='coerce')
Treasuries['Date'] = pd.to_datetime(Treasuries['Date'])
rf_data['Date'] = pd.to_datetime(rf_data['Date'])

# Merge risk-free rates
data = pd.merge(data, rf_data, on='Date', how='left')
data['DTS'] = data['spread'] * data['duration']

# Split data into 3 buckets - low, medium, high DTS subportfolios
bucket_labels = [1, 2, 3]
data['Bucket'] = pd.qcut(data['DTS'], q=len(bucket_labels), labels=bucket_labels, duplicates='drop')

# Filter out extreme spread points - keep middle 99.5% of the data
lower_quantile = data['DTS'].quantile(0.005)
upper_quantile = data['DTS'].quantile(0.995)
data = data[(data['DTS'] > lower_quantile) & (data['DTS'] < upper_quantile)]

# Filter so spread > 30
data = data[data['spread'] > 30 / 10000]

# Define target DTS by bucket (based on given values)
low_thresh = data['DTS'].quantile(0.33)
mid_thresh = data['DTS'].quantile(0.67)
high_thresh = data['DTS'].quantile(1.0)
target_dts = {1: low_thresh / 2, 2: (low_thresh + mid_thresh) / 2, 3: (mid_thresh + high_thresh) / 2}
print(target_dts)

# Group trading data by Date and Bucket
trading = data.groupby(['Date', 'Bucket']).apply(lambda x: x).reset_index(drop=True)

# Optimizer for Portfolio Weights
def portfolio_optimizer(portfolio, target_dts, initial_weights, previous_weights=None):
    n = len(portfolio)

    def objective(weights):
        excess_returns = portfolio['ytm'] - portfolio['RF']
        risk_adjusted_return = np.dot(weights, excess_returns)

        # Increase penalty for weight concentration
        concentration_penalty = 0.05 * np.sum(weights ** 2)

        # Increase penalty for high portfolio turnover
        if previous_weights is not None and len(previous_weights) == len(weights):
            turnover_penalty = 0.05 * np.sum(np.abs(weights - previous_weights))
        else:
            turnover_penalty = 0

        return -risk_adjusted_return + concentration_penalty + turnover_penalty

    # Constraints for the optimization problem
    constraints = [
        {'type': 'eq', 'fun': lambda w: target_dts - np.dot(w, portfolio['DTS'])},
        {'type': 'eq', 'fun': lambda w: np.sum(w) - 1}  # Weights must sum to 1
    ]

    # Bounds for weights
    bounds = [(0, 1) for _ in range(n)]

    # Using previous weights or evenly distributed initial weights
    if initial_weights is None or len(initial_weights) != n:
        initial_weights = np.ones(n) / n  # Default equal weights if not given or not matching the portfolio size

    # Run the optimizer
    result = minimize(objective, initial_weights, bounds=bounds, constraints=constraints)
    return result.x if result.success else initial_weights  # Return initial_weights if optimization fails


def hedge_duration(portfolio_duration, treasury_data, previous_hedge=None):
    remaining_duration = portfolio_duration
    hedge_results = []
    total_hedge_return = 0  # To accumulate the total return from the hedging instruments

    # Calculate cost per unit of duration for each treasury
    treasury_data['cost_to_duration'] = treasury_data['closing_price'] / treasury_data['modified_duration']
    treasury_data = treasury_data.sort_values(by='cost_to_duration')

    # Track the used quantities from the previous hedge
    if previous_hedge is not None:
        previous_hedge_dict = {treasury['treasury_tool']: treasury['used_quantity'] for _, treasury in previous_hedge.iterrows()}
    else:
        previous_hedge_dict = {}

    for _, treasury in treasury_data.iterrows():
        if abs(remaining_duration) < 1e-6:
            break

        tool_duration = treasury['modified_duration']
        price = treasury['closing_price']
        prev_price = treasury['prev_price']

        # Determine the quantity needed for the current duration
        used_quantity = remaining_duration / tool_duration

        # Adjust the quantity based on the previous day's usage
        prev_quantity = previous_hedge_dict.get(treasury['CUSIP'], 0)
        quantity_change = used_quantity - prev_quantity

        if abs(quantity_change) > 0:
            # Calculate return for the treasury bond
            treasury_return = (price - prev_price) / prev_price if prev_price else 0

            # Calculate the weighted return based on the used quantity change
            weighted_return = quantity_change * treasury_return

            # Append hedge details to the results
            hedge_results.append({
                'treasury_tool': treasury['CUSIP'],
                'used_quantity': quantity_change,
                'cost': quantity_change * price,
                'return': weighted_return  # Add return to the results
            })

        # Update remaining duration to get closer to zero
        remaining_duration -= used_quantity * tool_duration

    # Add total hedge return to the final results
    hedge_results.append({'Total_Hedge_Return': total_hedge_return})

    return pd.DataFrame(hedge_results)

# Calculate Portfolio Metrics
def calculate_portfolio_metrics(sub_data, treasury_hedge, previous_cost, previous_hedge_cost):
    capital_gains = (sub_data['Weight'] * (sub_data['closing_price'] - sub_data['prev_price'])).sum()
    hedge_cost = treasury_hedge['cost'].sum() if not treasury_hedge.empty else 0
    hedge_returns = treasury_hedge['return'].sum() if not treasury_hedge.empty else 0
    portfolio_cost = (sub_data['Weight'] * sub_data['closing_price']).sum()

    if previous_cost is not None:
        prev_hedge_cost = previous_hedge_cost['cost'][0]
        coupon_income = previous_cost * (sub_data['Weight'] * sub_data['coupon_rate']).sum()
        total_return = (previous_cost + prev_hedge_cost) * (coupon_income + capital_gains - hedge_cost + hedge_returns) / (portfolio_cost + hedge_cost)
        excess_return = total_return - (sub_data['RF'] * (portfolio_cost)).sum()
    else:
        coupon_income = 0
        total_return = 0
        excess_return = 0

    return {
        'coupon_income': coupon_income,
        'capital_gains': capital_gains,
        'hedge_cost': hedge_cost,
        'portfolio_cost': portfolio_cost,
        'total_return': total_return,
        'excess_return': excess_return
    }

def trading_and_hedging(trading_data, treasury_data, target_dts):
    results = []

    # Sort trading data by Date and CUSIP
    trading_data = trading_data.sort_values(by=['CUSIP', 'Date'])
    trading_data['prev_price'] = trading_data.groupby('CUSIP')['closing_price'].shift(1)
    treasury_data['prev_price'] = treasury_data.groupby('CUSIP')['closing_price'].shift(1)

    # Initialize previous weights as None at the beginning
    previous_weights = {}
    previous_hedges = {}
    previous_cost = {}

    for date, daily_data in trading_data.groupby('Date'):
        daily_treasury_data = treasury_data[treasury_data['Date'] == date]

        for bucket in [1, 2, 3]:
            sub_data = daily_data[daily_data['Bucket'] == bucket]
            target_dts_value = target_dts.get(bucket, None)

            if target_dts_value is None or len(sub_data) == 0:
                continue

            # Get previous weights or assign equal initial weights
            initial_weights = previous_weights.get(bucket, None)

            # Optimized weights using previous weights as initial guess
            optimized_weights = portfolio_optimizer(sub_data[['DTS', 'ytm', 'closing_price', 'RF']], target_dts_value, initial_weights, previous_weights.get(bucket))

            # Save the current optimized weights for the next iteration
            previous_weights[bucket] = optimized_weights

            sub_data = sub_data.copy()
            sub_data['Weight'] = optimized_weights
            portfolio_duration = (sub_data['Weight'] * sub_data['modified_duration']).sum()

            # Adjust the hedge based on previous hedge positions
            previous_hedge = previous_hedges.get(bucket, None)
            treasury_hedge = hedge_duration(portfolio_duration, daily_treasury_data, previous_hedge)

            metrics = calculate_portfolio_metrics(sub_data, treasury_hedge, previous_cost.get(bucket), previous_hedges.get(bucket))
            metrics['date'] = date
            metrics['bucket'] = bucket
            metrics['rf_rate'] = sub_data['RF'].mean()
            results.append(metrics)

            # Save the current hedge for the next iteration
            previous_hedges[bucket] = treasury_hedge
            previous_cost[bucket] = metrics['portfolio_cost']

    results_df = pd.DataFrame(results)
    return results_df

# Execute Trading and Hedging
results = trading_and_hedging(trading, Treasuries, target_dts)

results

C:\Users\hanna\anaconda3\lib\site-packages\pyfolio\pos.py:26: UserWarning: Module "zipline.assets" not found; mutltipliers will not be applied to position notionals.
  warnings.warn(


{1: 0.007506729735063554, 2: 0.02869297774205911, 3: 0.19642182898566762}


,coupon_income,capital_gains,hedge_cost,portfolio_cost,total_return,excess_return,date,bucket,rf_rate
0,0.000000,0.000000,2.402919,100.721128,0.000000,0.000000,2022-05-18,1,0.001
1,0.000000,0.000000,5.351416,98.650057,0.000000,0.000000,2022-05-18,2,0.001
2,0.000000,0.000000,14.502309,94.387951,0.000000,0.000000,2022-05-18,3,0.001
3,2.082298,0.154250,-0.265294,101.121610,2.557952,-0.576818,2022-05-25,1,0.001
4,2.886862,0.112217,-0.915174,99.033699,4.148431,0.187083,2022-05-25,2,0.001
...,...,...,...,...,...,...,...,...,...
373,3.614204,-0.033451,-12.319218,97.506407,20.821681,-55.233317,2024-11-06,2,0.020
374,3.013474,-1.552656,29.062552,70.247179,-19.271047,-64.229242,2024-11-06,3,0.020
375,5.330372,0.057681,-7.389142,96.193708,15.635603,-36.308999,2024-11-13,1,0.020
376,3.631793,0.048026,14.222083,97.596810,-8.022574,-88.051958,2024-11-13,2,0.020


In [2]:
results.to_excel('Data/DTSResults.xlsx', index = False)

In [31]:
results_agg = pl.DataFrame(results).with_columns((pl.col('rf_rate') * pl.col('portfolio_cost')).alias('rf_return'))
agg = results_agg.group_by('bucket').agg(pl.col('excess_return').sum(), pl.col('total_return').sum()).sort('bucket')

sharpes = []
for bucket in [1, 2, 3]:
    sharpe_ratio = pf.timeseries.sharpe_ratio(results_agg.filter(pl.col('bucket') == bucket)['total_return'].to_pandas(), risk_free = results_agg.filter(pl.col('bucket') == bucket)['rf_return'].mean(), period = 'weekly')
    sharpes.append(sharpe_ratio)

agg = agg.with_columns(pl.Series(name="sharpe_ratio", values=sharpes))

agg.to_pandas().set_index('bucket')

Signature: pf.timeseries.sharpe_ratio(returns, risk_free=0, period='daily')
Source:   
@deprecated(msg=DEPRECATION_WARNING)
def sharpe_ratio(returns, risk_free=0, period=DAILY):
    """
    Determines the Sharpe ratio of a strategy.

    Parameters
    ----------
    returns : pd.Series
        Daily returns of the strategy, noncumulative.
        - See full explanation in :func:`~pyfolio.timeseries.cum_returns`.
    risk_free : int, float
        Constant risk-free return throughout the period.
    period : str, optional
        Defines the periodicity of the 'returns' data for purposes of
        annualizing. Can be 'monthly', 'weekly', or 'daily'.
        - Defaults to 'daily'.

    Returns
    -------
    float
        Sharpe ratio.
    np.nan
        If insufficient length of returns or if if adjusted returns are 0.

    Note
    -----
    See https://en.wikipedia.org/wiki/Sharpe_ratio for more details.
    """

    return ep.sharpe_ratio(returns, risk_free=risk_free, period=perio

# DTS-NEUTRAL Duration-TARGET PORTFOLIO OPTIMIZATION

In [51]:
# upload and clean data
cdx_data = pd.read_excel("Data/TradingCDS.xlsx")
cdx_data['Date'] = pd.to_datetime(cdx_data['Date'])
cdx_data['DTS'] = cdx_data['Spread_Duration']
cdx_data = cdx_data.sort_values(by=['Type', 'Tenor', 'Date'])
cdx_data['prev_price'] = cdx_data.groupby(['Type', 'Tenor'])['Price'].shift(1)
cdx_data['prev_price'] = cdx_data['prev_price'].fillna(cdx_data['Price'])

# Load trading data
data = pd.read_excel("Data/TradingData.xlsx")

# Load treasury data
Treasuries = pd.read_excel("Data/TradingTreasuries.xlsx")

# Load risk-free rate (RF) data
rf_data = pd.read_csv("Data/rf_constant.csv")
rf_data.rename(columns={"Week_Start": "Date"}, inplace=True)

# Format datetime columns
data['Date'] = pd.to_datetime(data['Date'])
data['maturity_date'] = pd.to_datetime(data['maturity_date'], errors='coerce')
data['next_call_date'] = pd.to_datetime(data['next_call_date'], errors='coerce')
Treasuries['Date'] = pd.to_datetime(Treasuries['Date'])
rf_data['Date'] = pd.to_datetime(rf_data['Date'])

# Merge risk-free rates
data = pd.merge(data, rf_data, on='Date', how='left')
data['DTS'] = data['spread'] * data['duration']

# Filter out extreme spread points - keep middle 99.5% of the data
lower_quantile = data['modified_duration'].quantile(0.005)
upper_quantile = data['modified_duration'].quantile(0.995)
data = data[(data['modified_duration'] > lower_quantile) & (data['modified_duration'] < upper_quantile)]

data = data.sort_values(by=['CUSIP', 'Date'])
data['prev_price'] = data.groupby('CUSIP')['closing_price'].shift(1)
data['prev_price'] = data['prev_price'].fillna(data['closing_price'])

# Partition bonds into duration buckets
data['Bucket_Duration'] = pd.qcut(data['modified_duration'], q=3, labels=[1, 2, 3], duplicates='drop')

# Target durations
low_thresh_duration = np.quantile(data['modified_duration'], 0.33)
mid_thresh_duration = np.quantile(data['modified_duration'], 0.67)
high_thresh_duration = np.quantile(data['modified_duration'], 1.0)

target_duration = {
    1: low_thresh_duration / 2,
    2: (low_thresh_duration + mid_thresh_duration) / 2,
    3: (mid_thresh_duration + high_thresh_duration) / 2,
}
print("Target Duration:", target_duration)

# Group trading data by Date and Bucket_Duration
trading_by_duration = data.groupby(['Date', 'Bucket_Duration']).apply(lambda x: x).reset_index(drop=True)

def hedge_dts(portfolio_dts, cdx_data, previous_hedge=None):
    """
    Hedge a portfolio's DTS using CDX instruments based on Type and Tenor.
    """
    remaining_dts = portfolio_dts
    hedge_results = []
    total_hedge_return = 0

    # Calculate cost per DTS for sorting
    cdx_data['cost_to_dts'] = cdx_data['Price'] / cdx_data['DTS'].replace(0, np.nan)
    cdx_data = cdx_data.dropna(subset=['cost_to_dts']).sort_values(by='cost_to_dts')

    # Prepare previous hedge dictionary
    if previous_hedge is not None and not previous_hedge.empty:
        previous_hedge_dict = {
            (hedge['Type'], hedge['Tenor']): hedge['used_quantity']
            for _, hedge in previous_hedge.iterrows()
        }
    else:
        previous_hedge_dict = {}

    # Iterating through CDX instruments
    for _, cdx in cdx_data.iterrows():
        if abs(remaining_dts) < 1e-6:
            break

        tool_dts = cdx['DTS']
        price = cdx['Price']
        prev_price = cdx.get('prev_price', price)  # Use prev_price if available, else price

        cdx_key = (cdx['Type'], cdx['Tenor'])
        used_quantity = remaining_dts / tool_dts
        prev_quantity = previous_hedge_dict.get(cdx_key, 0)
        quantity_change = used_quantity - prev_quantity

        if abs(quantity_change) > 0:
            # Calculate return
            cdx_return = (price - prev_price) / prev_price if prev_price else 0
            weighted_return = quantity_change * cdx_return
            total_hedge_return += weighted_return

            # Append hedge details, including 'cost'
            hedge_results.append({
                'Date': cdx['Date'],
                'Type': cdx['Type'],
                'Tenor': cdx['Tenor'],
                'Price': price,
                'used_quantity': quantity_change,
                'cost': abs(quantity_change) * price, 
                'return': weighted_return,
            })

        remaining_dts -= used_quantity * tool_dts

    # Append total hedge return at the end
    hedge_results.append({'Date': None, 'Total_Hedge_Return': total_hedge_return})

    return pd.DataFrame(hedge_results)

def calculate_portfolio_metrics(sub_data, cdx_hedge, previous_cost, previous_hedge_cost):
    capital_gains = (sub_data['Weight'] * (sub_data['closing_price'] - sub_data['prev_price'])).sum()
    hedge_cost = cdx_hedge['cost'].sum() if not cdx_hedge.empty else 0
    hedge_returns = cdx_hedge['return'].sum() if not cdx_hedge.empty else 0
    portfolio_cost = (sub_data['Weight'] * sub_data['closing_price']).sum()

    if previous_cost is not None:
        prev_hedge_cost = previous_hedge_cost['cost'][0]
        coupon_income = previous_cost * (sub_data['Weight'] * sub_data['coupon_rate']).sum()
        total_return = (previous_cost + prev_hedge_cost) * (coupon_income + capital_gains - hedge_cost + hedge_returns) / (portfolio_cost + hedge_cost)
        excess_return = total_return - (sub_data['RF'] * (portfolio_cost)).sum()
    else:
        coupon_income = 0
        total_return = 0
        excess_return = 0

    return {
        'coupon_income': coupon_income,
        'capital_gains': capital_gains,
        'hedge_cost': hedge_cost,
        'portfolio_cost': portfolio_cost,
        'total_return': total_return,
        'excess_return': excess_return
    }

def trading_and_hedging_duration(bond_data, cdx_data, target_duration):
    results = []
    previous_weights = {}
    previous_hedges = {}
    previous_cost = {}

    for date, daily_bond_data in bond_data.groupby('Date'):
        daily_cdx_data = cdx_data[cdx_data['Date'] == date]

        for bucket in [1, 2, 3]:
            sub_data = daily_bond_data[daily_bond_data['Bucket_Duration'] == bucket].copy()

            target_duration_value = target_duration.get(bucket, None)
            if target_duration_value is None or len(sub_data) == 0:
                continue

            initial_weights = previous_weights.get(bucket, None)
            optimized_weights = portfolio_optimizer(
                sub_data[['DTS', 'modified_duration', 'ytm', 'closing_price', 'RF']],
                target_duration_value, initial_weights
            )
            previous_weights[bucket] = optimized_weights

            sub_data['Weight'] = optimized_weights
            portfolio_dts = (sub_data['Weight'] * sub_data['DTS']).sum()

            previous_hedge = previous_hedges.get(bucket, None)
            cdx_hedge = hedge_dts(portfolio_dts, daily_cdx_data, previous_hedge)
            previous_hedges[bucket] = cdx_hedge

            metrics = calculate_portfolio_metrics(sub_data, cdx_hedge, previous_cost.get(bucket), previous_hedges.get(bucket))
            metrics['date'] = date
            metrics['bucket'] = bucket
            metrics['rf_rate'] = sub_data['RF'].mean()
            results.append(metrics)

            previous_cost[bucket] = metrics['portfolio_cost']

    return pd.DataFrame(results)

results_duration = trading_and_hedging_duration(trading_by_duration, cdx_data, target_duration)

results_duration

Target Duration: {1: 0.8443787872791291, 2: 3.5015277683734896, 3: 12.009909296035765}


,coupon_income,capital_gains,hedge_cost,portfolio_cost,total_return,excess_return,date,bucket,rf_rate
0,0.000000,0.000000,3620.990521,98.091791,0.000000,0.000000,2022-05-18,1,0.001
1,0.000000,0.000000,13358.341781,99.710178,0.000000,0.000000,2022-05-18,2,0.001
2,0.000000,0.000000,86887.399018,92.481624,0.000000,0.000000,2022-05-18,3,0.001
3,9.185618,0.064360,605.904249,98.063728,-596.642429,-600.172723,2022-05-25,1,0.001
4,3.677505,0.701329,2619.529439,100.372985,-2614.359337,-2619.879851,2022-05-25,2,0.001
...,...,...,...,...,...,...,...,...,...
373,4.322728,-0.250301,12633.572117,97.664866,-12629.143132,-12679.928862,2024-11-06,2,0.020
374,3.478187,-0.794959,41322.938204,87.026274,-41319.745020,-41399.809192,2024-11-06,3,0.020
375,4.339488,0.031360,974.671610,93.009509,-970.435498,-1037.402345,2024-11-13,1,0.020
376,4.316520,0.389536,4049.266765,98.054402,-4044.154205,-4095.142494,2024-11-13,2,0.020


In [52]:
results_agg_duration = pl.DataFrame(results_duration).with_columns((pl.col('rf_rate') * pl.col('portfolio_cost')).alias('rf_return'))
agg_duration = results_agg_duration.group_by('bucket').agg(pl.col('excess_return').sum(), pl.col('total_return').sum()).sort('bucket')

sharpes_duration = []
for bucket in [1, 2, 3]:
    filtered_data = results_agg_duration.filter(pl.col('bucket') == bucket)
    sharpe_ratio_duration = pf.timeseries.sharpe_ratio(filtered_data['total_return'].to_pandas(), risk_free = filtered_data['rf_return'].mean(), period = 'weekly')
    sharpes_duration.append(sharpe_ratio_duration)

agg_duration = agg_duration.with_columns(pl.Series(name="sharpe_ratio", values=sharpes_duration))

agg_duration.to_pandas().set_index('bucket')

,excess_return,total_return,sharpe_ratio
bucket,,,
1,-7.181004e+05,-7.099373e+05,-3.567048
2,-3.271066e+06,-3.263175e+06,-3.824083
3,-1.441067e+07,-1.440279e+07,-3.837425
